# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
 

# Constants (Changed based on your data)

In [3]:
if name == 'yannis':
    sigma_yield = 490
    EModulus = 65241.07 
    nu = 0.34 
elif name == 'fabian':
    sigma_yield = 490
    EModulus = 65420.46
    nu = 0.34
elif name == 'daniel':
    sigma_yield = 490
    EModulus = 65669.47
    nu = 0.34
elif name == 'felix':
    sigma_yield = 490
    EModulus = 65669.47
    nu = 0.34

#Panel constants 
length = 750
width = 200


# Open stress values & properties of panels 

In [4]:
#Imoport panel stress file and reformat 
paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
paneldf = paneldf.reset_index()
#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

print(paneldf.head(5))
print(panelPropertiesdf.head(5))

   Element ID Component Name  sigmaXX  sigmaYY  sigmaXY        Load Case
0           1         panel1   -65.78   12.580    25.90  Subcase 1 (LC1)
1           2         panel1   -81.87   20.590    26.74  Subcase 1 (LC1)
2           3         panel1   -66.73   20.470    36.48  Subcase 1 (LC1)
3           4         panel2   -96.38    2.383    34.06  Subcase 1 (LC1)
4           5         panel2   -88.32   25.010    26.68  Subcase 1 (LC1)
   Element ID  thickness     mass
0           1          4  0.00054
1           2          4  0.00054
2           3          4  0.00054
3           4          4  0.00054
4           5          4  0.00054


# Split up the loadcases 

In [5]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case,thickness,mass
Element ID,,,,,,,
1,panel1,-65.78,12.580,25.90,Subcase 1 (LC1),4,0.00054
2,panel1,-81.87,20.590,26.74,Subcase 1 (LC1),4,0.00054
3,panel1,-66.73,20.470,36.48,Subcase 1 (LC1),4,0.00054
4,panel2,-96.38,2.383,34.06,Subcase 1 (LC1),4,0.00054
5,panel2,-88.32,25.010,26.68,Subcase 1 (LC1),4,0.00054


# Now we need to combine into panels where we have columns:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [ ]:
# We can just average the stresses
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

panelLC1

TypeError: type str doesn't define __round__ method

# Now we can apply the functions to the load cases 

In [ ]:
panelLC1['Reserve Factor'] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand' )
panelLC2['Reserve Factor'] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1 )
panelLC3['Reserve Factor'] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1 )
panelLC2

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Reserve Factor
Component Name,,,,,,,
panel1,-73.090000,-17.880000,29.706667,4.0,750,200,0.96
panel10,-73.090000,-17.880000,29.706667,4.0,750,200,0.96
panel2,-92.500000,-17.881333,29.610000,4.0,750,200,0.82
panel3,-93.896667,-17.879000,29.610000,4.0,750,200,0.81
panel4,-93.710000,-17.880000,30.066667,4.0,750,200,0.81
panel5,-93.256667,-17.879333,52.546667,4.0,750,200,0.75
panel6,-93.256667,-17.879333,52.546667,4.0,750,200,0.75
panel7,-93.710000,-17.880000,30.066667,4.0,750,200,0.81
panel8,-93.896667,-17.879000,29.610000,4.0,750,200,0.81
